# Agenda

1. Lots of questions
2. Even more questions!

# What are the methods that all Python objects have?

- Everything in Python is an object
- Everything has a class (type)
- Everything has attributes as well (things that come after a .)

- Methods are attributes
- Through inheritance, objects have access to methods on their classes, and also anything that their class inherits from

- Everyone eventually inherits from `object`, the ultimate top object in the Python hierarchy.

So the only methods that *everything* in Python has access to are those that are defined on `object`, and which we then get via inheritance.

What are those?

- `__new__` -- this creates new objects.  You should *NOT EVER EVER EVER* define this method unless you *really* know what you're doing.
- `__init__` -- this adds new attributes to objecst. This is typically overridden in a class you create.
- `__str__` -- this returns a string representation of your object, meant for end users
- `__repr__` -- this returns a string representation of your object, meant for programmers.

There are a few other methods as well, such as `__setattr__` and `__getattr__` which are kind of low level.

You can always see what methods (and attributes in general) are available to an object with the `dir` function, which returns a list of strings.

# List comprehensions and confusion that they can cause

Comprehensions allow you to create a new list based on an existing iterable, by describing what you want in the resulting iterable.  They're very similar in many ways to SQL.

When should you use a list comprehension? When you have a source iterable, you want to create a list, and you can describe an expression that maps from the source to the destination.

You want to use a regular `for` loop if you're not interested in the resulting loop, but are interested in the assignments/changes/side effects that take place in the loop.  For example: printing and writing to a file.


In [1]:
[one_number**2                  # any expression -- kind of like SELECT in SQL
 for one_number in range(10)]   # any iteration -- kind of like FROM in SQL

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [2]:
[one_number**2                  # any expression -- kind of like SELECT in SQL
 for one_number in range(10)    # any iteration -- kind of like FROM in SQL
 if one_number % 2 == 0]        # any condition -- kind of like WHERE in SQL

[0, 4, 16, 36, 64]

# How can we redefine `*` to do multiplication on our own custom objects?  How does this work?

All operators in Python are translated into method calls.  So when you say `2 + 3`, that's turned into `int.__add__(2, 3)`.  And when you say `2 * 3`, that's translated into `int.__mul__(2, 3)`.

In [3]:
10 + '3'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [4]:
'3' + 10

TypeError: can only concatenate str (not "int") to str

In [16]:
class MyData:
    def __init__(self, x):
        self.x = x
        
    def __mul__(self, other):
        print(f'Multiplying {self=} by {other=}')    # New in 3.8's f-strings!
        if hasattr(other, 'x'):
            return self.x * other.x
        else:
            return self.x * int(other)
        
    def __rmul__(self, other):
        print(f'Multiplying {self=} by {other=}, but reversed!')
        return self * other
    
a = MyData(10)
b = MyData(3)

a * b

Multiplying self=<__main__.MyData object at 0x11406e640> by other=<__main__.MyData object at 0x11406efa0>


30

In [17]:
# What happens if we multiply by something that lacks an x attribute?
a * 6

Multiplying self=<__main__.MyData object at 0x11406e640> by other=6


60

In [18]:
6 * a

Multiplying self=<__main__.MyData object at 0x11406e640> by other=6, but reversed!
Multiplying self=<__main__.MyData object at 0x11406e640> by other=6


60

# Top tips for becoming a great Python programmer

"Great Python programmer" == "fluent"

1. Use it a lot! Write lots of Python code. Make lots of mistakes. Get better.
2. Get feedback from others on Python code. (If you can work on a open-source project, that often does wonders.)  Pair programming!
3. Practice practice practice.
4. Read about programming, programming techniques.
5. Watch + listen to conference videos.

# Testing of `input` is hard!

If you're using `pytest`, then you can use the `monkeypatch` facility to rewire `sys.stdin` to any other file. Then you have control over the input.  Better yet, you can use a `io.StringIO`, which implements the same API as `sys.stdin`, but is in memory.  So you can set things up with that.



# I overwrote the type! How do I get it back?

I'm going to assume that you redefined `type` in your program.  You should assign to anything in builtins. However, it's hard to notice sometimes. Pay attention to what PyCharm/VSCode/PyLint tells you about this!

If you do, then you can usually rewrite the code.  You don't want redefined builtins lying around in your code.



# What does -O do in terms of optimizing code? 

The `-O` option does almost nothing.  I think that it only turns `__debug__` from `True` to `False`. It also removes `assert` statements.  And it removes docstrings if you use `-OO`.

# How to use `pytest`?

First: `pytest` is amazing! 

Second: Get Brian's book (from the Pragmatic Programmers) about `pytest`.  

Third: I have some archived Linux Journal columns about `pytest`.  I think that I wrote three articles about that.

# Loops: `for` vs. `while`

Loops are part of the whole "DRY" idea in programming: Don't Repeat Yourself!

- If code repeats itself several lines in a row, use a loop.
- if it repeats itself in several places in your program, use a function.
- If it repeats itself in several programs, use a module.

How to decide between `for` and `while`?

- `for` loops are for when you want to do something with every element in a sequence. Add every number, check every IP address, spam every customer.
- `while` loops stop when we reach a condition, not when we get through a sequence.  So if you don't know how many times you're going to do something, but you know when you want to stop, that's a `while` loop.

In [19]:
mylist = [10, 20, 30, 40, 50]

total = 0
for one_item in mylist:
    total += one_item
    
total    

150

In [20]:
total = 0
while total < 60:
    s = input('Enter a number: ').strip()
    
    if not s.isdigit():
        print(f'{s} is not numeric! Try again!')
        continue  # go back for another iteration
        
    print(f'Adding {s} to total')
    total += int(s)
    
print(f'Exited the loop; {total=}')        
    

Enter a number: 5
Adding 5 to total
Enter a number: 20
Adding 20 to total
Enter a number: 30
Adding 30 to total
Enter a number: 2
Adding 2 to total
Enter a number: 8
Adding 8 to total
Exited the loop; total=65


# High level overview of async (aka `asyncio`)

When I have a network server handling many clients, I'll typically do one of the following two things:

1. One process with many threads, and every thread handles a separate client. Problem is that threads in Python aren't really concurrent, so only one thread is running at a time.  Also, threads will drive you insane.
2. Many processes, each handling a separate client. Problem here is that every process is completely insulated from the others, and this takes much more memory + CPU.

A new approach, `asyncio`, has had ups and downs, but seems to be gaining lots of fans. It has a totally different way of doing things -- very similar to JavaScript's `nodejs`:

3. Have one process, and one thread. Then have a whole bunch of functions in a list. Iterate over the list, one function at a time, and ask the function to run for a little while.  The function goes to sleep after running a bit.  Each function handles a network client, so if you have 500 clients, you'll have 500 functions on your list. Keep iterating over them, going back to the beginning when you reach the end, until the list is empty.

Is async really parallel? No. Is it great in all situations? No.  Does it speed many things up? YES, because in many cases, the network latency and/or the time needed by the other side of the network is much greater than the time it takes to come back to each function.



# How does Python know when to use `__repr__`?

`__str__ is meant for end users.  Meaning, it's used whenever we use `print` or `str` on something.

`__repr__` is meant for developers. It's really only used by Jupyter, debuggers, the `repr` builtin function, or some weird cases -- such as an object inside of a list.

- If you only define `__repr__`, then it covers for `__str__` as needed.
- If you only define `__str__`, then it doesn't cover `__repr__`, meaning you'll get `object.__repr__`, which is *SUPER* ugly!

In [21]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'[str] Person with {self.name=}'
    
    def __repr__(self):
        return f'[repr] Person with {self.name=}'
    
p = Person('Reuven')

In [22]:
print(p)

[str] Person with self.name='Reuven'


In [23]:
str(p)

"[str] Person with self.name='Reuven'"

In [24]:
p

[repr] Person with self.name='Reuven'

In [25]:
[p, p, p]

[[repr] Person with self.name='Reuven',
 [repr] Person with self.name='Reuven',
 [repr] Person with self.name='Reuven']

# Learning advanced Python topics

1. Fluent Python
2. My own courses: Weekly Python Exercise, B series (intermediate/advanced)
3. Anthony Shaw's book, "CPython Internals"
4. Descriptors book by Jacob Zimmerman

# Remembering how to set up a new class

1. Every class should have a `__init__` method, which is where you set up the data (attributes) for your class.
2. After a while, it'll become second nature

*BUT*

You can also try dataclasses (as of Python 3.7), which make it every easier and simpler!

In [27]:
from dataclasses import dataclass

@dataclass
class Person:
    first : str   # type hints on class attributes!
    last : str
    shoesize : int
        
p1 = Person('Reuven', "Lerner", 46)        
p2 = Person('John', 'Doe', 47)

In [28]:
print(p1)

Person(first='Reuven', last='Lerner', shoesize=46)


In [29]:
print(p2)

Person(first='John', last='Doe', shoesize=47)


In [30]:
p1.first

'Reuven'

In [31]:
p2.first

'John'

# Dict comprehensions

Just as list comprehensions create lists, dict comprehensions create dictionaries. (Not a list of dictionaries!)

In [32]:
words = 'This is a bunch of words for my Python course'.split()

# dict comprehension!
{ one_word : len(one_word) # key : value for each pair in our new dict
  for one_word in words}   # iterating over my list, words

{'This': 4,
 'is': 2,
 'a': 1,
 'bunch': 5,
 'of': 2,
 'words': 5,
 'for': 3,
 'my': 2,
 'Python': 6,
 'course': 6}

In [33]:
def count_vowels(s):
    total = 0
    for one_letter in s.lower():
        if one_letter in 'aeiou':
            total += 1
            
    return total

{ one_word : count_vowels(one_word) # key : value for each pair in our new dict
  for one_word in words}   # iterating over my list, words

{'This': 1,
 'is': 1,
 'a': 1,
 'bunch': 1,
 'of': 1,
 'words': 1,
 'for': 1,
 'my': 0,
 'Python': 1,
 'course': 3}

For more about comprehensions, see here: https://lerner.co.il/2015/07/16/want-to-understand-pythons-comprehensions-think-like-an-accountant/

# Learning Python by doing

1. My book, "Python Workout" (https://PythonWorkout.com/)
2. My exercise courses, Weekly Python Exercise (https://WeeklyPythonExercise.com/)
3. PyBytes (https://pybytes.com/)
4. Python Morsels (https://PythonMorsels.com/)
5. Manning has LiveProjects

# Suggested ways to learn regular expressions

I have a free e-mail course: https://RegexpCrashCourse.com/, that in 14 e-mail segments, sent daily, will teach you regular expression in general, using Python.

Also check out https://regular-expressions.info/, which has info about regular expressions, including comparisons.

# Is TensorFlow a library in Python?

I don't know very much about it, but I think it's written in C or C++, and has bindings in Python.  TensorFlow is a library for Deep Learning (a type of machine learning).

# Subsets



In [36]:
set(dir(object)).issubset(set(dir(x)))

True

In [37]:
# create a set
s1 = {10, 20, 30}

s1.issubsetset(s1)

True

In [38]:
s1 <= s1  # same thing as issubset!

True

# How did I get multiplication to run by renaming `__radd__` to `__rmul__`?

Simple answer: I made a mistake!  `__add__` -> `__radd__` (for reversed addition), but we were doing multiplication, so `__mul__` -> `__rmul__`

# Engaging with Python APIs

If it's a Web API, I like to use `requests`

For creating APIs, I've heard that FastAPI is amazing.

I used Flask for a simple API I needed to create last year, and it took me 2 hours, including setting up a PostgreSQL database.

# `exec` and `eval`

Python has *statements* and *expressions*.  Statements don't return values, but do things (e.g., assignment).  Expressions return values.

`eval` lets you take a string containing a Python expression, and you get the value back.

`exec` takes a string containing one or more Python stattements, and executes them.

Both are super dangerous!  

I like to say that there's a 75% overlap between `eval` and `evil`.  If you're getting input from the user, or a database, or just about anywhere, and you use `eval`, then you should be very afraid.

# Coroutines vs. functions

In asyncio, we write special functions that know how to go to sleep, and can be put on a loop.  The loop is known as the "event loop" and when they go to sleep, our functions are said to `await`. For a variety of reasons, functions running on the event loop are known as "coroutines."

# How to get started with open source?

1. Find a project you care about.
2. They are definitely looking for help!
3. Offer to help!
4. Learn how to use GitHub's pull requests and such
5. Get lots of feedback, and do more and more over time.
6. If you hate it -- the people, the project, etc. -- leave and find another project.

https://www.firsttimersonly.com/

# Good sites other than Python Tutor

1. https://PythonTutor.com/ (I have a YouTube video introducing it)
2. https://pythex.org/ (regular expression debugger)

# Learning about decorators

Check out my PyCon 2019 talk, "Practical Decorators," on YouTube!

# What concept or logic did I learn after a long time?

1. Attributes -- I keep hammering at teaching attributes because they're so central to Python and no one talks about them.  (Get deeper with properties and descriptors.)
2. I keep learning more about machine learning and algorithms

# Other languages

I used to do a ton of Perl, then a ton of Ruby (and some JavaScript).

Right now, I'm doing 95% Python + a tiny bit of SQL.  

I want to learn Rust.  But I don't have time to get past a few pages in the book I bought a few years ago.

